In [2]:
import requests
from bs4 import BeautifulSoup

def pedido(limit=100, page=1,job_id=None):
    if job_id:
        url = f"https://api.itjobs.pt/job/get.json?api_key=ee176fa9456283ab9c42f357b036e236&id={job_id}"
    else:
        url = f"https://api.itjobs.pt/job/list.json?api_key=ee176fa9456283ab9c42f357b036e236&limit={limit}&page={page}"
    payload = {}
    headers = {'User-Agent': "ALPCD_5"}  # Necessário pôr 'User-Agent' nos headers
    res = requests.request("GET", url, headers=headers, data=payload)
    if res.status_code == 200:  # Verificar se o acesso foi bem sucedido (200 OK)
        results = res.json()
        return results
    else:
        print(f"Erro {res.status_code} - {res.text}")
        return {}

job_data = pedido()
limit = 100
page = 1
res = pedido(limit, page)
response = res
list_results= response["results"]
#soup = BeautifulSoup(res.text, "lxml")

while page*limit < response["total"]:
    page += 1
    new_results = pedido(limit, page)["results"]
    list_results += new_results
response["results"] = list_results


In [ ]:
import time
from datetime import datetime

sorted_results = sorted(list_results, key=lambda x: x['updatedAt'], reverse= True)
'''
up2 = 'sim'
for sort in sorted_results:
    up = sort["updatedAt"][:10]
    if up == up2:
        continue
    else:
        up2 = sort["updatedAt"][:10]
'''
given_skills = ['english']
start_date = '2024-11-03'       #PERGUNTAR AO PROFESSOR SE É NECESSARIO ACEITAR HORA TAMBEM
end_date = '2024-11-03'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')


filtered_results = []
for res in sorted_results:
    update_date_str = res['updatedAt'][:10]
    update_date = datetime.strptime(update_date_str, '%Y-%m-%d')
    if start_date <= update_date <= end_date:
        filtered_results.append(res)
print(f"A analisar {len(filtered_results)} resultados.")


A analisar 21 resultados.


In [45]:
import re

skills = ['[english,', 'Python]']
joined_skills = ' '.join(skills).lower()
skills = re.findall(r'[^\[\],\s]+', joined_skills)
# 


for res in list_results:
    loc = res['updatedAt']
    print(loc)



2024-11-08 22:25:14
2024-11-08 22:24:51
2024-11-08 22:24:48
2024-11-08 22:24:43
2024-11-08 22:24:39
2024-11-08 22:24:34
2024-11-08 22:24:30
2024-11-08 22:24:25
2024-11-08 22:24:21
2024-11-08 22:23:49
2024-11-08 22:23:46
2024-11-08 22:23:42
2024-11-08 22:23:39
2024-11-08 22:23:36
2024-11-08 22:23:23
2024-11-08 22:23:17
2024-11-08 22:23:16
2024-11-08 22:23:02
2024-11-08 22:24:00
2024-11-08 22:22:54
2024-11-08 22:22:50
2024-11-08 22:22:37
2024-11-08 22:21:49
2024-11-08 22:21:36
2024-11-08 12:13:45
2024-11-08 12:12:19
2024-11-08 12:11:57
2024-11-08 12:11:19
2024-11-08 12:11:14
2024-11-08 12:11:11
2024-11-08 12:11:08
2024-11-08 12:11:05
2024-11-08 12:11:02
2024-11-08 12:10:57
2024-11-08 12:10:54
2024-11-08 12:10:50
2024-11-08 12:10:47
2024-11-08 12:10:37
2024-11-08 12:10:35
2024-11-08 12:10:27
2024-11-08 12:10:07
2024-11-08 12:09:29
2024-11-08 12:09:15
2024-11-07 20:14:42
2024-11-07 20:14:37
2024-11-07 20:14:28
2024-11-07 20:14:24
2024-11-07 20:14:19
2024-11-07 20:14:12
2024-11-07 20:14:07


In [ ]:
# imports
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def filter_by_dates_results(list_results, start_date, end_date, given_skills):
    sorted_results = sorted(list_results, key=lambda x: x["updatedAt"], reverse=True)
    filtered_results = []
    count = 0
    print("aqui1")
    for res in sorted_results:
        update_date_str = res['updatedAt']
        update_date = datetime.strptime(update_date_str, '%Y-%m-%d %H:%M:%S')
        if update_date <= end_date:
            if start_date <= update_date:
                count += 1
                print(count, end=', ')
                res = process_job(res, given_skills)
                filtered_results.append(res)
            else:
                break
    print("aqui2")
    return filtered_results

def skills(given_skills, start_date, end_date, list_results):
    start_date, end_date = start_date.replace(' ',''), end_date.replace(' ','')

    if end_date < start_date:
        print(f"Data final menor que inicial, a iniciar com valores trocados...")
        start_date, end_date = end_date, start_date

    if len(start_date) == 10:
        start_date += " 00:00:01"
    if len(end_date) == 10:
        end_date += " 23:59:59"

    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        print("Erro ao ler as datas, verifique se estas existem.")
        return {}
    
    filtered_results = filter_by_dates_results(list_results, start_date, end_date, given_skills)

    for result in filtered_results:
        if result:
            print(result)
        else:
            filtered_results.remove(result)

def process_job(res, given_skills):
    body = res['body']

    try:
        annotations = skill_extractor.annotate(body)
        
    except IndexError as e:
        print(f"Erro a 'anotar' o 'body': {body}")
        raise e
    except (IndexError, ValueError) as e:
        print(f"Erro a processar o 'body': {e}")
        return None

    annoted_skills = [skill['doc_node_value'].lower() for skill in annotations['results']['full_matches']]
    annoted_skills += [skill['doc_node_value'].lower() for skill in annotations['results']['ngram_scored']]

    if set(given_skills) & set(annoted_skills):      # all(item in annoted_skills for item in given_skills)
        return res
    else:
        return None

'''
def process_jobs_concurrently(list_of_results, given_skills, max_workers=4):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_res = {executor.submit(process_job, res, given_skills): res for res in list_of_results}

        for future in as_completed(future_to_res):
            res = future.result()
            if res:
                results.append(res)
                print(f"Processed job ID: {res['id']}")

    return results
'''
# iniciar o skill extractor
nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# Given skills, start date, and end date
given_skills = ['english']
start_date = '2024-11-03'      
end_date = '2024-11-03     '

skills(given_skills, start_date, end_date, list_results)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...
aqui1
aqui2


# d)

In [3]:
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def pedido(limit, page):
    url = f"https://api.itjobs.pt/job/list.json?api_key=ee176fa9456283ab9c42f357b036e236&limit={limit}&page={page}"
    payload = {}
    headers = {'User-Agent': "ALPCD_5"}  # Necessário por 'User-Agent' nos headers
    res = requests.request("GET", url, headers=headers, data=payload)
    if res.status_code == 200:  # Verificar se o acesso foi bem sucedido (200 OK)
        results = res.json()
        return results
    else:
        print(f"Erro {res.status_code} - {res.text}")
        return {}
    
def fetch_data():
    global list_results  # Para ser acessado dentro de outras funções
    limit = 100
    page = 1
    response = pedido(limit, page)
    list_results = response["results"]
    print("A recolher respostas do URL...")

    while page * limit < response["total"]:  # Para limite=100 e total=1261, página vai até 13
        page += 1
        try:  # Caso seja devolvido um dicionário vazio, para não ocorrerem erros
            new_results = pedido(limit, page)["results"]
            list_results += new_results  # Lista com todos os resultados que vai incrementando
        except:
            print(f"Erro ao obter resultados da página {page}.")

    response["results"] = list_results  # Finalmente cria o 'response' com todos os resultados
    print("Finalizado.")

def filter_by_dates_results(list_results, start_date, end_date):
    sorted_results = sorted(list_results, key=lambda x: x["updatedAt"], reverse=True)
    filtered_results = []

    for res in sorted_results:
        update_date_str = res['updatedAt']
        update_date = datetime.strptime(update_date_str, '%Y-%m-%d %H:%M:%S')
        if update_date <= end_date:
            filtered_results.append(res)
        if update_date < start_date:
            break
    
    print(len(filtered_results))
    return filtered_results

def skills(given_skills, start_date, end_date):
    if not list_results:
        fetch_data()

    # Iniciar o skill extractor
    nlp = spacy.load("en_core_web_lg")
    skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

    start_date, end_date = start_date.replace(' ',''), end_date.replace(' ','')

    if end_date < start_date:
        print(f"Data final menor que inicial, a iniciar com valores trocados...")
        start_date, end_date = end_date, start_date

    if len(start_date) == 10:
        start_date += " 00:00:01"
    if len(end_date) == 10:
        end_date += " 23:59:59"

    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        print("Erro ao ler as datas, verifique se estas existem.")
        return {}
    
    filtered_results = filter_by_dates_results(list_results, start_date, end_date)

    results = process_jobs_concurrently(filtered_results, given_skills, skill_extractor, max_workers=8)
    
    print(results)

def process_job(res, given_skills, skill_extractor):
    body = res['body']
    try:
        annotations = skill_extractor.annotate(body)
    except (IndexError, ValueError) as e:
        print(f"Erro a processar o 'body': {e}")
        return None

    annoted_skills = [skill['doc_node_value'] for skill in annotations['results']['full_matches']]
    annoted_skills += [skill['doc_node_value'] for skill in annotations['results']['ngram_scored']]

    if set(given_skills) & set(annoted_skills):  
        return res
    return None

def process_jobs_concurrently(list_of_results, given_skills, skill_extractor, max_workers=4):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_res = {executor.submit(process_job, res, given_skills, skill_extractor): res for res in list_of_results}

        for future in as_completed(future_to_res):
            res = future.result()
            if res:
                results.append(res)

    return results

# Given skills, start date, and end date
given_skills = ['english']
start_date = '2024-11-03'       
end_date = '2024-11-03'

skills(given_skills, start_date, end_date)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...
22


c:\Users\games\AppData\Local\Programs\Python\Python312\Lib\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


Erro a processar o 'body': 'special' is not in list
Erro a processar o 'body': 'special' is not in list
[{'id': 491865, 'company': {'id': 6731, 'name': 'YellowIpe', 'logo': 'https://static.itjobs.pt/images/companies/0e/915/6731/logo.png', 'description': 'YellowIpe® is your IT professional services consultancy. We offer the best tailored solutions for the requirements and challenges of your technological project.', 'url': 'https://yellowipe.io', 'url_facebook': 'https://www.facebook.com/yellowipe.io', 'url_linkedin': 'https://www.linkedin.com/company/yellowipe/', 'slug': 'yellowipe'}, 'companyId': 6731, 'title': 'Technical Writer', 'body': '<p><strong>About YellowIpe</strong><br /><br />Our mission is to inspire the connection between technology and people, we foster the best of our professionals through our expertise in finding and attracting the best talent for the best projects. The Focus on People, Collaboration and Commitment are the pillars that guide us in this trajectory.<br /><

In [ ]:
import re
import time
import typer
import requests

app = typer.Typer()

def pedido(limit, page, job_id = None):
    if job_id:
        url = f"https://api.itjobs.pt/job/get.json?api_key=ee176fa9456283ab9c42f357b036e236&id={job_id}"
    else:
        url = f"https://api.itjobs.pt/job/list.json?api_key=ee176fa9456283ab9c42f357b036e236&limit={limit}&page={page}"
    payload = {}
    headers = {'User-Agent': "ALPCD_5", 'Cookie': 'itjobs_pt=3cea3cc1f4c6a847f8c459367edf7143:94de45f2a55a15b2672adf8788ac8072e7bfd5c5'}  # Necessário por 'User-Agent' nos headers
    res = requests.request("GET", url, headers=headers, data=payload)
    if res.status_code == 200:  # Verificar se o acesso foi bem sucedido (200 OK)
        results = res.json()
        return results
    else:
        print(f"Erro {res.status_code} - {res.text}")
        return {}

# Comando para extrair o salário de um job específico
def salary(job_id: str):
    # Usar a função com retentativa para obter os detalhes do job
    job_data = pedido(100,1,job_id)
    
    if job_data is None:
        print("Não foi possível obter os dados do job. Verifique o job_id e tente novamente.")
        return
    
    # Verificar se o campo wage está presente e tem valor
    wage = job_data.get("wage")
    if wage:
        print(f"Salário encontrado: {wage}")
    else:
        # Caso wage esteja vazio ou seja None, procurar por valores salariais em outros campos
        description = job_data.get("description", "")
        matches = re.findall(r"\b\d{1,3}(?:\.\d{3})*(?:,\d{2})?\b", description)
        
        if matches:
            # Exibe o primeiro valor encontrado que pode representar um salário
            salary_estimate = matches[0]
            print(f"Salário estimado encontrado na descrição: {salary_estimate}")
        else:
            print("Salário não especificado na oferta de emprego.")

salary(491881)

Salário encontrado: 60000
